# Processamento de Linguagem Natural - Faculdade Descomplica
# Aula 7 - Agrupamento de Documentos e Nuvem de Palavras

Prof. Elvis de Souza

# Processando o dataset

In [ ]:
import pandas as pd

In [ ]:
dataset = pd.read_csv("https://github.com/alvelvis/data-movies-willianoliveiragibin/raw/main/data-willianoliveiragibin.csv", index_col=0)

In [ ]:
dataset

In [ ]:
dataset.Description

In [ ]:
type(dataset.Description[0])

In [ ]:
dataset.Description = dataset.Description.apply(eval)

In [ ]:
type(dataset.Description[0])

In [ ]:
dataset.Description = dataset.Description.apply(lambda x: " ".join(x))

In [ ]:
dataset.Description

# Extraindo os tokens do dataset

In [ ]:
import string
punctuation = string.punctuation

import nltk
nltk.download("punkt")
nltk.download("stopwords")

from nltk.tokenize import word_tokenize

stopwords = nltk.corpus.stopwords.words("english")

In [ ]:
def extrair_tokens(texto):
    freq = {}
    tokens = word_tokenize(texto)
    tokens = [token.lower() for token in tokens if token not in punctuation and token.lower() not in stopwords]
    return tokens

In [ ]:
dataset['tokens'] = dataset.Description.apply(extrair_tokens)

In [ ]:
dataset.tokens[0]

In [ ]:
todos_os_tokens = []
for tokens in dataset.tokens:
    todos_os_tokens.extend(tokens)

In [ ]:
print(len(todos_os_tokens))

In [ ]:
print(len(set(todos_os_tokens)))

In [ ]:
freq_tokens = {}
for token in todos_os_tokens:
    if not token in freq_tokens:
        freq_tokens[token] = 0
    freq_tokens[token] += 1

tokens_mais_frequentes = sorted(freq_tokens.items(), key=lambda x: -x[1])[:1000]

In [ ]:
tokens_mais_frequentes

In [ ]:
tokens_mais_frequentes = [x[0] for x in tokens_mais_frequentes]
print(tokens_mais_frequentes)

# Vetorizando todas as descrições de filmes

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def vetorizar(tokens):
    vetor = []
    for token in tokens_mais_frequentes:
        vetor.append(tokens.count(token))
    return np.asarray(vetor)

In [ ]:
primeiro_filme = dataset.tokens[0]

In [ ]:
vetor_primeiro_filme = vetorizar(primeiro_filme)

In [ ]:
print(vetor_primeiro_filme)
print(len(vetor_primeiro_filme))

In [ ]:
dataset['vetores'] = dataset['tokens'].apply(vetorizar)

In [ ]:
dataset.vetores

# Definindo função para achar os vetores mais próximos de um filme

In [ ]:
def top_vetores(nome):
    indice = dataset[dataset['Movie Name'] == nome].index[0]
    similaridades = cosine_similarity(dataset.vetores[indice].reshape(1, -1), list(dataset.vetores))
    ranking = sorted([(dataset['Movie Name'][i], similaridade) for i, similaridade in enumerate(similaridades[0])], key=lambda x: x[1], reverse=True)
    return ranking

In [ ]:
dataset['Movie Name']

In [ ]:
print(top_vetores("The Godfather"))

In [ ]:
str(dataset.Description[1])

In [ ]:
str(dataset.Description[1098])

# Vídeo 4: Contagem de palavras para nuvem de palavras



In [ ]:
dataset.tokens

In [ ]:
def contar_tokens(tokens):
    freq_tokens = {}
    for token in tokens:
        if not token in freq_tokens:
            freq_tokens[token] = 1
        else:
            freq_tokens[token] += 1
    return freq_tokens

In [ ]:
dataset['freq_tokens'] = dataset['tokens'].apply(contar_tokens)

In [ ]:
dataset.freq_tokens

# Construindo nuvem de palavras

In [ ]:
! pip3 install wordcloud

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
titulos = ["The Godfather", "The Lord of the Rings: The Fellowship of the Ring", "Spider-Man: Across the Spider-Verse"]

In [ ]:
print(top_vetores("The Godfather"))

In [ ]:
grupos = []
for titulo in titulos:
    vetores_proximos = [x[0] for x in top_vetores(titulo)]
    grupos.append(vetores_proximos[:5])

In [ ]:
grupos

In [ ]:
for grupo in grupos:
    todas_frequencias = {}
    for filme in grupo:
        todas_frequencias.update(dataset[dataset['Movie Name'] == filme].iloc[0].freq_tokens)

    wordcloud = WordCloud(background_color='white')
    wordcloud.generate_from_frequencies(frequencies=todas_frequencias)
    plt.figure()
    plt.title(grupo[0])
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.show()